In [11]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size = 0.2, random_state = 42)

def model_fn(a_layer = None):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  model.add(keras.layers.Dense(100, activation='relu'))
  if a_layer:
    model.add(a_layer)
  model.add(keras.layers.Dense(10, activation='softmax'))
  return model

model = model_fn()
model.summary()
'''
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
history = model.fit(train_scaled, train_target, epochs = 20, verbose = 0)        #verbose = 0 -> 훈련과정 출력 X

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

adam = keras.optimizers.Adam(learning_rate = 0.1  )
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
history = model.fit(train_scaled, train_target, epochs = 20, verbose = 0,
                    validation_data =(val_scaled, val_target))
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()
'''
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
history = model.fit(train_scaled, train_target, epochs = 10, verbose = 0, validation_data = (val_scaled, val_target))
model.save_weights('model-weights.h5')      #모델의 파라미터 저장
model.save('model-whole.h5')                #모델 구조와 파라미터 저장

model = model_fn(keras.layers.Dropout(0.3))
model.load_weights('model-weights.h5')

import numpy as np
val_labels = np.argmax(model.predict(val_scaled), axis= -1)
print(np.mean(val_labels == val_target))

model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5', save_best_only = True)           #ModelCheckpoint  save_best_only -> 가장 낮은 검증 손실 점수를 낸 모델 저장 (에포크 횟수)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)       #에포크를 끝까지 수행하지않고 patience 값 연속으로 검증 손실 점수 가 향상되지 않으면 그대로 중지
model.fit(train_scaled, train_target, epochs = 10, verbose = 1, validation_data = (val_scaled, val_target), callbacks = [checkpoint_cb, early_stopping_cb])




Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_25 (Flatten)        (None, 784)               0         
                                                                 
 dense_50 (Dense)            (None, 100)               78500     
                                                                 
 dense_51 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
375/375 [==============================] - 1s 1ms/step
0.8778333333333334
Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5971 - accuracy: 0.7896 - val_loss: 0.4439 - val_accuracy: 0.8394
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4404 - accuracy: 